In [198]:
import requests
from json import loads as json_loads, dumps as json_dumps
from time import sleep
import pprint

In [179]:
generic_try_again_error_msg = 'Beklenmeyen bir hata oluştu, lütfen kısa bir süre içerisinde tekrar deneyiniz.'

In [180]:
phone_number = input("phone")

In [181]:
s = requests.session()

In [182]:
# Get sms verification code
payload = {
  "recipient": phone_number
}
sms_verification_response = s.post("https://vatandas.iys.org.tr/api/v1/cbff/recipients", data=payload)
try:
    if json_loads(sms_verification_response.content)["error"]["summary"] == generic_try_again_error_msg:
        print("Lütfen 1 dakika bekleyin..")
except KeyError:
    oid = json_loads(sms_verification_response.content.decode())["payload"]["id"]

In [ ]:
# Get token with sms verification code
sms_code = input("=>")
login_response = s.post("https://giris.iys.org.tr/auth/realms/citizen/protocol/openid-connect/token", data= {
    "client_id": "e-devlet",
    "grant_type": "password",
    "recipient": phone_number,
    "oid": oid,
    "code": sms_code
})
token = json_loads(login_response.content)["access_token"]
print(token)
login_response.headers

In [185]:
# Authorizated header
headers = {
    'Authorization': f'Bearer {token}',
}

In [ ]:
# Get brand list
payload = json_dumps({
  "recipient": phone_number
})
_link = "https://vatandas.iys.org.tr/api/v1/cgw/recipients/BIREYSEL/brands/search/"
list_response = s.post(_link, headers=headers, data=payload)
brand_list = json_loads(list_response.content)["list"]
list_response

In [ ]:
headers = {
  'Authorization': f'Bearer {token}',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json',
  'DNT': '1',
  'Origin': 'https://vatandas.iys.org.tr',
  'Referer': 'https://vatandas.iys.org.tr/izinlerim/onay-ret',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-GPC': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
  'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"'
}
for brand in brand_list:
    reach_type = brand["type"]
    brand_code = brand["brandCode"]
    if brand["status"] == "ONAY":
        
        url = f"https://vatandas.iys.org.tr/api/v1/cgw/recipients/BIREYSEL/brands/{brand_code}/consent/update"
        payload = json_dumps({
            "recipient": phone_number,
            "status": "RET",
            "type": reach_type
            })
        response = s.request("POST", url, headers=headers, data=payload)
        sleep(3)
        if response.status_code != 200:
            print("Error with the transaction")
            break
        print(brand["brandCode"],brand["brandName"], brand["type"], "RET")
